notebook này sẽ giúp cậu crawl data tiếng việt cho dự án máy tìm kiếm động vật

Trước tiên cần có 1 từ điển tên động vật, ở đây tôi sử dụng trang https://animalia.bio để lấy danh sách này </br>sau khi tìm kiếm trên google với tiếng anh, thử chuyển đổi qua tiếng việt nếu được, nếu không, lưu lại log

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  
      
# Mở trình duyệt
chrome_options = Options()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome( options=chrome_options)
driver.set_window_position(-1920, 0)
driver.maximize_window()

from list_name import crawl_list_name 
number_each_group=crawl_list_name(driver)
print(number_each_group)


In [14]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup
from crawl_data.wiki_content import crawl_content_wiki
from find_link import find_wiki_en, find_wiki_vn

import os
path=os.getcwd()
#lấy từ đầu đến thư mục animal-search-engine
path=path[:path.find("animal-search-engine")+len("animal-search-engine")]
print(path)
os.chdir(path)

async def process_animal(session, animal, couit):
    check, wiki_en_url = await find_wiki_en(session, animal)
    if check:
        check, wiki_vn_url = await find_wiki_vn(session, animal, wiki_en_url)
        if check:
            await crawl_content_wiki(session, wiki_vn_url, couit)


async def main():
    with open("crawl_data/log/check_log.txt", "r", encoding="utf-8") as f:
        last_line = f.readlines()[-1].strip()
    
    check = False
    animalds = ["mammals", "mollusk", "birds", "amphibia", "reptiles"]
    couit = 0
    
    connector = aiohttp.TCPConnector(limit=50)  # Giới hạn kết nối đồng thời
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = []
        for ani in animalds:
            with open(f"crawl_data/list_name/{ani}.txt", "r", encoding="utf-8") as f:
                animals = [line.strip() for line in f.readlines()]
            
            inner_count = 0
            #Loop 
            for animal in animals:
                if animal == last_line or last_line == "PhamPhong":
                    check = True
                if not check:
                    couit += 1
                    continue
                
                task = await process_animal(session, animal, couit)
                tasks.append(task)
                couit += 1
                inner_count += 1
                
                if inner_count == 5:
                    await asyncio.gather(*tasks)
                    tasks = []
                    inner_count = 0
                
                with open("crawl_data/log/check_log.txt", "w", encoding="utf-8") as f:
                    f.write(animal)


result = await main()

p:\coddd\animal-search-engine
Unexpected error on attempt 1 for https://en.wikipedia.org/wiki/Roze's_gecko: name 'BeautifulSoup' is not defined
Unexpected error on attempt 1 for https://en.wikipedia.org/wiki/Slater's_worm_lizard: name 'BeautifulSoup' is not defined
Unexpected error on attempt 1 for https://en.wikipedia.org/wiki/Oreosaurus_serranus: name 'BeautifulSoup' is not defined
Unexpected error on attempt 1 for https://en.wikipedia.org/wiki/Pholidobolus_paramuno: name 'BeautifulSoup' is not defined


TypeError: An asyncio.Future, a coroutine or an awaitable is required